In [86]:
#Defining packages to use
import requests as rq
import pandas as pd
import json
from IPython.display import JSON 
import os
import matplotlib
import seaborn

##Import csv qith city bike data
city_bike_data = pd.read_csv('data/cityBikeData.csv')

# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [87]:
#Get Foursquare API key
api_key_foursquare = os.environ["FOURSQUARE_API_KEY"]
#Set headers for URL
headers_fsq = { "accept": "application/json",
            "Authorization": api_key_foursquare}

In [88]:
#Function to get data from Foursquare
def get_foursquare_data(data,headers):
    #Empty to get values from results processing
    foursquare_data =[]
    for i,r in data.iterrows():
        
        #Get latitude and longitude to get the parameter ll for the API
        lt = r['latitude']
        lg = r['longitude']
       
        #URL to call places in a radius of 1000m and category 13000 that is dinning and drinking places
        url = f"https://api.foursquare.com/v3/places/search?radius=1000&categories=13000&ll={lt},{lg}"
        
        #Get results in json
        results_json       = rq.request('GET',url, headers = headers)
        foursquare_results = results_json.json()     

      #Get results in json
     
        for row in foursquare_results['results']:
           #the json tag for results is called results
           foursquare_data.append({
               'fsq_id'    :row['fsq_id'],
               'bk_station_distance' :row['distance'],
               'latitude'  :row['geocodes']['main']['latitude'],
               'longitude' :row['geocodes']['main']['longitude'],
               'name'      :row['name'],
               'category_name':row['categories'][0]['name'],
               'uid': r['uid']
               }) 
        data_fsq = pd.DataFrame(foursquare_data)
     
    return data_fsq

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [89]:
# Call function to process CSV
df_foursquare = get_foursquare_data(city_bike_data,headers_fsq)
# Record dataframe results in CSV
df_foursquare.to_csv('data/foursquare_data.csv',index = False, sep= ',')

Put your parsed results into a DataFrame

In [90]:
#Show dataframe with results
df_foursquare

,fsq_id,bk_station_distance,latitude,longitude,name,category_name,uid
0,4a15b070f964a520b9781fe3,36,34.034828,-118.231642,Tony's Saloon,Cocktail Bar,bcycle_lametro_3037
1,55fccaa5498ec525f32b644d,134,34.034393,-118.232483,Everson Rocye Bar,Cocktail Bar,bcycle_lametro_3037
2,4fc2ae63e4b069481edde91f,132,34.034313,-118.229864,Bread Lounge,Bakery,bcycle_lametro_3037
3,5b549f2ec97f28002cb31a5f,88,34.034376,-118.231987,Guerrilla Tacos,Latin American Restaurant,bcycle_lametro_3037
4,60c184c9528f822158a147e8,93,34.035173,-118.232148,Afuri Ramen,Ramen Restaurant,bcycle_lametro_3037
...,...,...,...,...,...,...,...
2206,4a91e909f964a520511c20e3,579,34.060321,-118.444161,Espresso Profeta,Café,bcycle_lametro_4644
2207,5796b9ff498e154db94c0979,541,34.060612,-118.444520,Tender Greens,American Restaurant,bcycle_lametro_4644
2208,56e5adc6498e04218ff90bf1,353,34.061061,-118.446155,Cava,Restaurant,bcycle_lametro_4644
2209,56c3e0a1cd10d878ed8ebdc7,763,34.057892,-118.443051,Poke Me,American Restaurant,bcycle_lametro_4644


# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [91]:
#Get YELP API key from OS
api_key_yelp = os.environ["YELP_API_KEY"]


#Set headers for URL
headers_yelp = {
    "accept": "application/json",
    "Authorization": api_key_yelp
}


In [92]:
#Function to get data from Yelp
def get_yelp_data(data,headers):
    #Empty to get values from results processing
    yelp_data =[]
    for i,r in data.iterrows():
    
        #get latitude and longitude to get for the API
        lt = r['latitude']
        lg = r['longitude']
        #URL to call places in a radius of 1000m and category "bars and restaurants" that is dinning and drinking places
        url = f"https://api.yelp.com/v3/businesses/search?radius=1000&categories=restaurants&latitude=" + str(lt) + "&longitude="+ str(lg)
        
        #get results in json
        results_json       = rq.request('GET',url, headers = headers)
        yelp_results       = results_json.json()
       
       #Get results from json

        for row in yelp_results['businesses']:
           #the json tag for results is called results
           yelp_data.append({
               'yelp_id'    :row['id'],
               'bk_station_distance' :row['distance'],
               'latitude'  :row['coordinates']['latitude'],
               'longitude' :row['coordinates']['longitude'],
               'name'      :row['name'],
               'category_name': row.get('title', 'Restaurant'),
               'price'     :row.get('price', None),
               'rating'    :row.get('rating', None),
               'review_count': row.get('review_count',None),
               'uid': r['uid']
                }) 
        data_ylp = pd.DataFrame(yelp_data)
        
    return data_ylp

**Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)**

In [93]:
# Call function to process CSV
df_yelp = get_yelp_data(city_bike_data,headers_yelp)
# Record dataframe results in CSV
df_yelp.to_csv('data/yelp_data.csv',index = False, sep= ',')

**Put your parsed results into a DataFrame**

In [94]:
#Show data in dataframe
df_yelp

,yelp_id,bk_station_distance,latitude,longitude,name,category_name,price,rating,review_count,uid
0,fEY0zHaDMfIW3-N__joDKQ,209.717313,34.034020,-118.229190,Bestia,Restaurant,$$$,4.5,7244,bcycle_lametro_3037
1,0WyRlH-fxOVLh1b3oEBEEQ,884.469696,34.041974,-118.235426,Urth Caffé- Downtown LA,Restaurant,$$,4.0,5786,bcycle_lametro_3037
2,wo3wnj3pMSOPkzqpvbey-g,748.192703,34.041510,-118.232610,Bavel,Restaurant,$$$,4.5,2100,bcycle_lametro_3037
3,SNlf3YpKkDG-e4R9jkwLhA,87.197427,34.034456,-118.232130,Guerrilla Tacos,Restaurant,$$,4.0,1079,bcycle_lametro_3037
4,GVt0MwAoQcuM06V_-fHkug,277.659076,34.032417,-118.230381,The Lobos Truck,Restaurant,$,4.5,1201,bcycle_lametro_3037
...,...,...,...,...,...,...,...,...,...,...
4411,IkBr4b8ktf9L5ksxo9vtng,768.104557,34.057957,-118.443140,Poke Me,Restaurant,$$,4.0,715,bcycle_lametro_4644
4412,yZZOlnfE3w7G41WqRgeNiw,500.112425,34.060990,-118.444680,The Boiling Crab,Restaurant,$$$,3.5,970,bcycle_lametro_4644
4413,hdabpPPAaPp6Q3rQ47zZOw,315.937408,34.062344,-118.446648,Enzo's Pizzeria,Restaurant,$$,3.5,729,bcycle_lametro_4644
4414,SsQmelYytgCNxzBm0T1CCw,671.493972,34.060260,-118.442964,Toranj,Restaurant,$$,4.5,446,bcycle_lametro_4644


# Comparing Results

**Which API provided you with more complete data? Provide an explanation.**

> - The yelp API has more information about the locations but it does not have a good categories clasifications as Foursquare does.
> - The Foursquare api got the rating removed in their latest version.
> - So I think the most complete data is on Yelp.

**Get the top 10 restaurants according to their rating**

In [95]:
df_top10_yelp = df_yelp.sort_values('rating', ascending = False)
df_top10_yelp.head(10)

,yelp_id,bk_station_distance,latitude,longitude,name,category_name,price,rating,review_count,uid
620,beA_DlDpij8O9xK2B-NjEA,643.977122,34.035187,-118.255925,Mex Peru Gipsy,Restaurant,$$,5.0,1505,bcycle_lametro_3081
3379,Kz1Vn3-tSOugmToyuvAThA,470.088184,34.172430,-118.364440,Humble Bird,Restaurant,$$,5.0,212,bcycle_lametro_4599
3726,CcqraT0cuGKYEcZ1ri_kxg,887.283675,34.042734,-118.250779,BROKEN MOUTH | Lee's Homestyle,Restaurant,$$,5.0,1434,bcycle_lametro_3027
1753,beA_DlDpij8O9xK2B-NjEA,1270.380675,34.035187,-118.255925,Mex Peru Gipsy,Restaurant,$$,5.0,1505,bcycle_lametro_4512
1007,teOH2ATgXmnGGREl05EMbg,605.749346,34.077160,-118.264675,Roots & Rye,Restaurant,$$,5.0,315,bcycle_lametro_4220
1767,T3cqmVnThp8qA0JV7Z2D4w,340.893956,34.063860,-118.286750,Hanu Korean BBQ,Restaurant,None,5.0,674,bcycle_lametro_4304
1782,bdm3708B0OJW9xDdaKVH_A,230.797411,34.067421,-118.309526,Gol Tong Chicken,Restaurant,$$,5.0,1787,bcycle_lametro_4538
286,e0Vs3U-XkLkZ7EJ6JgPAYQ,587.471982,34.070333,-118.292176,Olivia,Restaurant,$$,5.0,119,bcycle_lametro_4401
1802,CcqraT0cuGKYEcZ1ri_kxg,496.215719,34.042734,-118.250779,BROKEN MOUTH | Lee's Homestyle,Restaurant,$$,5.0,1434,bcycle_lametro_4516
3598,teOH2ATgXmnGGREl05EMbg,24.060143,34.077160,-118.264675,Roots & Rye,Restaurant,$$,5.0,315,bcycle_lametro_4510
